# Answering Business Questions using SQL

<b>Data Model</b>

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

<b>Sample Data</b>

- Media related data was created using real data from an iTunes Library. It is possible for you to use your own iTunes Library to generate the SQL scripts, see instructions here.

- Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.).

- Sales information is auto generated using random data for a four year period.

## Initial Setup 

In [41]:
# import modules
import sqlite3
import pandas as pd

# return pandas dataframe of a sql query
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)
    
# executes an sql command using the sqlite module
def run_command(command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(command)
        
# calls the run_query() to return a list of all tables and views in the db
def show_tables():
    q = "SELECT name, type FROM sqlite_master WHERE type IN (\"table\", \"view\")"
    result = run_query(q)
    print(result)
    
# display the tables and views in the db
show_tables()

              name   type
0            album  table
1           artist  table
2         customer  table
3         employee  table
4            genre  table
5          invoice  table
6     invoice_line  table
7       media_type  table
8         playlist  table
9   playlist_track  table
10           track  table


## Selecting the Albums to Purchase 

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|      Artist Name     |  Genre  |
|:--------------------:|:-------:|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [42]:
# create view of the genres sold in the USA
usa_tracks_sold = """
CREATE VIEW usa_tracks_sold AS
    SELECT g.name genre, count(*) tracks_sold
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    INNER JOIN track t ON il.track_id = t.track_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    WHERE c.country = 'USA'
    GROUP BY g.name;
"""
run_command(usa_tracks_sold)

In [68]:
# display the results
q = """
SELECT * from usa_tracks_sold;
"""
run_query(q)

,genre,tracks_sold
0,Alternative,35
1,Alternative & Punk,130
2,Blues,36
3,Classical,4
4,Easy Listening,13
5,Electronica/Dance,5
6,Heavy Metal,3
7,Hip Hop/Rap,20
8,Jazz,14
9,Latin,22


In [67]:
# find the percentage of the tracks sold by genre
q = """
SELECT genre, 
       tracks_sold, 
       ROUND(CAST(tracks_sold AS FLOAT)/(SELECT count(tracks_sold) FROM usa_tracks_sold), 2) percentage_sold
FROM usa_tracks_sold
ORDER BY tracks_sold DESC;
"""

run_query(q)

,genre,tracks_sold,percentage_sold
0,Rock,561,33.00
1,Alternative & Punk,130,7.65
2,Metal,124,7.29
3,R&B/Soul,53,3.12
4,Blues,36,2.12
5,Alternative,35,2.06
6,Latin,22,1.29
7,Pop,22,1.29
8,Hip Hop/Rap,20,1.18
9,Jazz,14,0.82


<b>Conclusion:</b> So, it seems the most tracks sold in the USA is music from the Rock genre, followed by the Alternative & Punk genre and thirdly followed by the Metal genre. 

Out of the 4 artists above, the three artists we should choose would be Red Tone (7.65% of the Punk tracks are sold), Slim Jim Bites (2.12% of Blues tracks are sold) and finally just edging it out is the Meteor and the Girls artist as the Pop genre has sold 1.29% tracks in the USA whereas the artist Regal in the Hip Hop genre has only 1.18% of Hip Hop genre tracks sold in the USA.

## Analyzing Employee Sales Performance

### Context 

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [69]:
sales_performance = """
SELECT e.first_name || ' ' || e.last_name employee, e.hire_date, SUM(il.unit_price) total_sales
FROM employee e
INNER JOIN customer c ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON c.customer_id = i.customer_id
INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
GROUP BY employee;
"""

employee_sales = run_query(sales_performance)
employee_sales

,employee,hire_date,total_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


<b>Comment:</b> The employee that generated the most total sales is Jane Peacock with \$1731.51, followed by Margaret Park with \$1584 and lastly Steve Johnson has generated only \$1393.92. Despite Steve being last out of the three, we can see that Jane has being at the company the longest which might explain the higher sales and Steve having the lowest, as he was the lasted to be hired out of the three.

## Analyzing Sales by Country 

The next task is to analyze the sales data for customers from each different country.

In particular, we have been directed to calculate data, for each country, on the:

- Total number of customers
- Total value of sales
- Average value of sales per customer
- Average order value

For countries with only one customer, we will group these up into a group called 'Other' for simplicity.

In [75]:
customers_by_country = """
CREATE VIEW customers_by_country AS
    SELECT
        CASE
           WHEN (SELECT count(*) FROM customer where country = c.country) = 1 
               THEN "Other"
           ELSE c.country
        END AS country,
        c.customer_id,
        il.*
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
"""

run_command(customers_by_country)

In [78]:
q = """
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM customers_by_country
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );
"""

run_query(q)

,country,customers,total_sales,average_order,customer_lifetime_value
0,USA,13,1040.49,7.942672,80.037692
1,Canada,8,535.59,7.047237,66.948750
2,Brazil,5,427.68,7.011148,85.536000
3,France,5,389.07,7.781400,77.814000
4,Germany,4,334.62,8.161463,83.655000
5,Czech Republic,2,273.24,9.108000,136.620000
6,United Kingdom,3,245.52,8.768571,81.840000
7,Portugal,2,185.13,6.383793,92.565000
8,India,2,183.15,8.721429,91.575000
9,Other,15,1094.94,7.448571,72.996000


<b>Comment:</b> 
According to the above table, the best recommendation to the marketing team for potential growth, would be the country Czech Republic as their average order per customer and average order value is the highest.

But, we must understand there is a sample bias involved here as we have extremely small sample size for each country and this does not represent the entire market of that country. Thus, this must be ongoin

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.